Packages that are required for VectorDB and Embeddings

- !pip3 install qdrant_client

- !pip3 install -U sentence-transformers
    - Dependency with pytorch version so we have been using python version 3.9.6

# Import Libraries

In [16]:
# Python Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Optional Packages
from typing import List, Tuple, Dict
from tqdm import tqdm

# VD Client Packages
from qdrant_client import models, QdrantClient

# Word Embeddings 
from sentence_transformers import SentenceTransformer

In [ ]:
# Reading Dataset